In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.environ import * 
from pyomo.opt import SolverFactory

model = ConcreteModel()

rm = [i for i in range(1,168)]
plant = ['BA1','BA2','BA3','BA4','BA5', 'BA6','BA7','BA8','BA9','BA10','BA11','BA12','BA14']
elem = [i for i in range(1,26)]
prod = ['BA11','BA51','BA52','BA61','BA62','BA63','BA71','BA81','BA82','BA83','BA101','BA111','BA121','BA122','BA141','BA142']
links = [('BA1','BA2'),('BA2','BA1'),('BA2','BA3'),('BA2','BA4'),('BA3','BA1'),('BA3','BA2'),('BA3','BA14'),('BA4','BA5'),('BA4','BA6'),('BA7','BA9'),('BA7','BA14'),('BA9','BA8'),('BA9','BA12'),('BA9','BA14'),('BA9','BA11'),('BA10','BA9'),('BA11','BA8'),('BA11','BA7'),('BA11','BA10')]
model.links = Set(dimen = 2, initialize = links)
lenlink = [i for i in range(len(links))]
objs = []
to_plant = []
from_plant = []
identify_plant = {}
for k in plant:
    arr1 = []
    arr2 = []
    arr3 = []
    for j in prod:
        if len(j)==4:
            if j[0:3]==k:
                if k not in identify_plant.keys():
                    identify_plant[k] = [j]
                else:
                    identify_plant[k].append(j)
        if len(j)==5:
            if j[0:4]==k:
                if k not in identify_plant.keys():
                    identify_plant[k] = [j]
                else:
                    identify_plant[k].append(j)
    #for i in elem:
    for j in model.links:
        if j[1] == k:
            arr1.append(j)
        if j[0] == k:
            arr2.append(j)
    to_plant.append(arr1)
    from_plant.append(arr2)



RM_analysis1 = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\RM_analysis.csv'))
RM_analysis1 = np.delete(RM_analysis1, 0, axis=1)
RM_analysis = {}
RM_index=[]
for i in range(len(RM_analysis1)):
    for j in range(RM_analysis1.shape[1]):
        RM_analysis[i+1,j+1]=RM_analysis1[i][j]
        RM_index.append((i+1,j+1))
        
Availability = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\Availability.csv'))
Availability = np.delete(Availability, 0, axis=1)
Availability = {i+1:Availability[i] for i in range(len(Availability))}
#print(Availability)

prod = ['BA11','BA51','BA52','BA61','BA62','BA63','BA71','BA81','BA82','BA83','BA101','BA111','BA121','BA122','BA141','BA142']
Rev_prod = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\Rev_prod.csv'))
Rev_prod = np.delete(Rev_prod, 0, axis=1)
Rev_prod = {prod[i]:Rev_prod[i] for i in range(len(Rev_prod))}
#print(Rev_prod)

prod = ['BA11','BA51','BA52','BA61','BA62','BA63','BA71','BA81','BA82','BA83','BA101','BA111','BA121','BA122','BA141','BA142']
Demand = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\Demand.csv'))
Demand = np.delete(Demand, 0, axis=1)
Demand = {prod[i]:Demand[i] for i in range(len(Demand))}
#print(Demand)

Rev_mat = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\Rev_mat.csv'))
Rev_mat = np.delete(Rev_mat, 0, axis=1)
Rev_mat = {i+1:Rev_mat[i] for i in range(len(Rev_mat))}
#print(Rev_mat)

prod_losses = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\prod_losses.csv'))
prod_losses = np.delete(prod_losses, 0, axis=1)
prod_losses = {(prod[i], j+1):prod_losses[i][j] for j in range(len(elem)) for i in range(len(prod))}
#print(prod_losses)

prod_spec_max = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\prod_spec_max.csv'))
prod_spec_max = np.delete(prod_spec_max, 0, axis=1)
prod_spec_max = {(prod[i], j+1):prod_spec_max[i][j] for j in range(len(elem)) for i in range(len(prod))}
#print(prod_spec_max)

mat_deductions = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\mat_deductions.csv'))
mat_deductions = np.delete(mat_deductions, 0, axis=1)
mat_deductions = {(i+1, j+1):mat_deductions[i][j] for j in range(len(elem)) for i in range(len(rm))}
#print(mat_deductions)

elem_price = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\elem_price.csv'))
elem_price = np.delete(elem_price, 0, axis=1)
elem_price = {i+1:elem_price[0][i] for i in range(len(elem_price[0]))}
#print(elem_price)

Capacity = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\Capacity.csv'))
Capacity = np.delete(Capacity, 0, axis=1)
Capacity = {plant[i]:Capacity[i] for i in range(len(Capacity))}
#print(Capacity)

prod_spec_min = np.array(pd.read_csv(r'C:\Users\dkakkad\Desktop\CMU\MSProject\GAMS\prod_spec_min.csv'))
prod_spec_min = np.delete(prod_spec_min, 0, axis=1)
prod_spec_min = {(prod[i], j+1):prod_spec_min[i][j] for j in range(len(elem)) for i in range(len(prod))}
#print(prod_spec_min)

In [5]:
import timeit

start = timeit.default_timer()

model = ConcreteModel()

model.links = Set(dimen = 2, initialize = links)

#Variables
model.f1 = Var(rm, plant, within = NonNegativeReals, name = 'flow of raw materials', initialize = 0) 
model.f2 = Var(model.links, elem, within = NonNegativeReals, name = 'intermediate flow' , initialize = 0)
model.f3 = Var(prod, elem, within = NonNegativeReals, name = 'flow of elements for each product', initialize = 0)
model.g = Var(prod, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s1 = Var(rm, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s2 = Var(plant, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s3 = Var(prod, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s4 = Var(prod, elem, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s5 = Var(prod, elem, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s6 = Var(rm, plant, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s7 = Var(model.links, elem, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s8 = Var(model.links, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s9 = Var(model.links, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.y1 = Var(rm, within = Binary, name = 'flow of products', initialize = 0)
model.y2 = Var(plant, within = Binary, name = 'flow of products', initialize = 0)
model.y3 = Var(prod, within = Binary, name = 'flow of products', initialize = 0)
model.y4 = Var(prod, elem, within = Binary, name = 'flow of products', initialize = 0)
model.y5 = Var(prod, elem, within = Binary, name = 'flow of products', initialize = 0)
model.y6 = Var(rm, plant, within = Binary, name = 'flow of products', initialize = 0)
model.y7 = Var(model.links, elem, within = Binary, name = 'flow of products', initialize = 0)
model.y8 = Var(model.links, within = Binary, name = 'flow of products', initialize = 0)

#parameters
model.RM_analysis = Param(RM_index, initialize = RM_analysis)
model.Availability = Param(rm, initialize = Availability)
model.Rev_prod = Param(prod, initialize = Rev_prod)
model.Rev_mat = Param(rm, initialize = Rev_mat)
model.prod_losses = Param(prod, elem, initialize = prod_losses)
model.mat_deductions = Param(rm, elem, initialize = mat_deductions)
model.elem_price = Param(elem, initialize = elem_price)
model.Capacity = Param(plant, initialize = Capacity)
model.Demand = Param(prod, initialize = Demand)
model.prod_spec_max = Param(prod, elem, initialize = prod_spec_max)
model.prod_spec_min = Param(prod, elem, initialize = prod_spec_min)

#objective
@model.Objective()
def obj(model):
    rev_prod = 0
    rev_mat = 0 
    ML_rm = 0
    ML_prod = 0
    for i in prod:
        for j in elem:
            rev_prod += model.f3[i,j]*model.Rev_prod[i]
            ML_prod += model.f3[i,j]*model.prod_losses[i,j]*model.elem_price[j]
    for i in rm:
        for j in plant:
            rev_mat += model.f1[i,j]*model.Rev_mat[i]
            for k in elem:
                ML_rm += model.f1[i,j]*model.RM_analysis[i, k]*model.mat_deductions[i,k]*model.elem_price[k]

    return (ML_prod - ML_rm - rev_prod - rev_mat)

@model.Constraint(elem, plant)
def MB1(model, elem, plant):
    if plant in identify_plant.keys():
        return sum(model.f1[m,plant]*model.RM_analysis[m,elem] for m in rm) + sum(model.f2[k,elem] for k in model.links if k[1] == plant) == sum(model.f2[k,elem] for k in model.links if k[0] == plant) + sum(model.f3[k,elem] for k in identify_plant[plant])
    else:
        return sum(model.f1[m,plant]*model.RM_analysis[m,elem] for m in rm) + sum(model.f2[k,elem] for k in model.links if k[1] == plant) == sum(model.f2[k,elem] for k in model.links if k[0] == plant)

@model.Constraint(rm)
def avail(model, rm):
#     print(model.Availability.display())
    return sum(model.f1[rm,j] for j in plant) - model.Availability[rm][0] + model.s1[rm] == 0

@model.Constraint(plant)
def capacity(model, plant):
    return sum(model.f1[m,plant] for m in rm) + sum(model.f2[k,m] for m in elem for k in model.links if k[1] == plant) - model.Capacity[plant][0] + model.s2[plant] == 0

@model.Constraint()
def rm_constraint(model):
    return sum((model.f1[m,'BA5']+model.f1[m,'BA6']+model.f1[m,'BA8']+model.f1[m,'BA12']) for m in rm) == 0

@model.Constraint(prod)
def demand(model, prod):
    return model.Demand[prod][0] - sum(model.f3[prod,m] for m in elem) + model.s3[prod] == 0

@model.Constraint(prod)
def prodflow(model, prod):
    return sum(model.f3[prod,m] for m in elem) == model.g[prod]

@model.Constraint(prod, elem)
def prod_min(model, prod, elem):
    return model.prod_spec_min[prod,elem]*model.g[prod] - model.f3[prod, elem] + model.s4[prod,elem] == 0

@model.Constraint(prod, elem)
def prod_max(model, prod, elem):
    return model.f3[prod, elem] - model.prod_spec_max[prod,elem]*model.g[prod] + model.s5[prod,elem] == 0

@model.Constraint(rm, plant)
def lb_f1(model,rm,plant):
    return -model.f1[rm,plant] + model.s6[rm, plant] == 0 

@model.Constraint(lenlink, elem)
def lb_f2(model,lenlink,elem):
    return -model.f2[links[lenlink],elem] + model.s7[links[lenlink],elem] == 0 

@model.Constraint(lenlink)
def constraint1(model, lenlink):
    return -sum(model.f2[links[lenlink],m] for m in elem) + model.s8[links[lenlink]] == 0

@model.Constraint(rm)
def BigM1(model, rm):
    return model.s1[rm] <= 1e8*(1 - model.y1[rm])

@model.Constraint(plant)
def BigM2(model, plant):
    return model.s2[plant] <= 1e8*(1 - model.y2[plant])

@model.Constraint(prod)
def BigM3(model, prod):
    return model.s3[prod] <= 1e8*(1 - model.y3[prod])

@model.Constraint(prod,elem)
def BigM4(model, prod,elem):
    return model.s4[prod,elem] <= 1e8*(1 - model.y4[prod,elem])

@model.Constraint(prod,elem)
def BigM5(model, prod,elem):
    return model.s5[prod,elem] <= 1e8*(1 - model.y5[prod,elem])

@model.Constraint(rm,plant)
def BigM6(model, rm,plant):
    return model.s6[rm,plant] <= 1e8*(1 - model.y6[rm,plant])

@model.Constraint(lenlink,elem)
def BigM7(model,lenlink,elem):
    return model.s7[links[lenlink],elem] <= 1e8*(1 - model.y7[links[lenlink],elem])

@model.Constraint(lenlink)
def BigM8(model, lenlink):
    return model.s8[links[lenlink]] <= 1e8*(1 - model.y8[links[lenlink]])

solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= False)
objs.append(-model.obj())

@model.Constraint()
def cut1(model):
    sum6,sum8 = 0,0
    for i in rm:
        for j in plant:
            if pyo.value(model.s6[i,j]) >= 1:
                sum6 += model.y6[i,j]
    for i in links:
        if pyo.value(model.s8[i]) >= 1:
            sum8 += model.y8[i]

    return sum8 >= 1

solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= False)
objs.append(-model.obj())

@model.Constraint()
def cut2(model):
    sum6,sum8 = 0,0
    for i in rm:
        for j in plant:
            if pyo.value(model.s6[i,j]) >= 1:
                sum6 += model.y6[i,j]
    for i in links:
        if pyo.value(model.s8[i]) >= 1:
            sum8 += model.y8[i]

    return sum8>=1
solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= False)
objs.append(-model.obj())

@model.Constraint()
def cut3(model):
    sum6,sum8 = 0,0
    for i in rm:
        for j in plant:
            if pyo.value(model.s6[i,j]) >= 1:
                sum6 += model.y6[i,j]
    for i in links:
        if pyo.value(model.s8[i]) >= 1:
            sum8 += model.y8[i]

    return sum8>=1
solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= False)
objs.append(-model.obj())

@model.Constraint()
def cut4(model):
    sum6,sum8 = 0,0
    for i in rm:
        for j in plant:
            if pyo.value(model.s6[i,j]) >= 1:
                sum6 += model.y6[i,j]
    for i in links:
        if pyo.value(model.s8[i]) >= 1:
            sum8 += model.y8[i]

    return sum8>=1
solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= False)
objs.append(-model.obj())

@model.Constraint()
def cut5(model):
    sum6,sum8 = 0,0
    for i in rm:
        for j in plant:
            if pyo.value(model.s6[i,j]) >= 1:
                sum6 += model.y6[i,j]
    for i in links:
        if pyo.value(model.s8[i]) >= 1:
            sum8 += model.y8[i]

    return sum8>=1
solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= False)
objs.append(-model.obj())

@model.Constraint()
def cut6(model):
    sum6,sum8 = 0,0
    for i in rm:
        for j in plant:
            if pyo.value(model.s6[i,j]) >= 1:
                sum6 += model.y6[i,j]
    for i in links:
        if pyo.value(model.s8[i]) >= 1:
            sum8 += model.y8[i]

    return sum8>=1
solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= False)
objs.append(-model.obj())

@model.Constraint()
def cut7(model):
    sum6,sum8 = 0,0
    for i in rm:
        for j in plant:
            if pyo.value(model.s6[i,j]) >= 1:
                sum6 += model.y6[i,j]
    for i in links:
        if pyo.value(model.s8[i]) >= 1:
            sum8 += model.y8[i]

    return sum8>=1
solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= False)
objs.append(-model.obj())

@model.Constraint()
def cut8(model):
    sum6,sum8 = 0,0
    for i in rm:
        for j in plant:
            if pyo.value(model.s6[i,j]) >= 1:
                sum6 += model.y6[i,j]
    for i in links:
        if pyo.value(model.s8[i]) >= 1:
            sum8 += model.y8[i]

    return sum8>=1
solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= False)
objs.append(-model.obj())

@model.Constraint()
def cut9(model):
    sum6,sum8 = 0,0
    for i in rm:
        for j in plant:
            if pyo.value(model.s6[i,j]) >= 1:
                sum6 += model.y6[i,j]
    for i in links:
        if pyo.value(model.s8[i]) >= 1:
            sum8 += model.y8[i]

    return sum8>=1
solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= False)
objs.append(-model.obj())

@model.Constraint()
def cut10(model):
    sum6,sum8 = 0,0
    for i in rm:
        for j in plant:
            if pyo.value(model.s6[i,j]) >= 1:
                sum6 += model.y6[i,j]
    for i in links:
        if pyo.value(model.s8[i]) >= 1:
            sum8 += model.y8[i]

    return sum8>=1
solver = SolverFactory('gams:cplex')
model.sol = solver.solve(model,tee= True)
objs.append(-model.obj())

stop = timeit.default_timer()

print('Time: ', stop - start)

--- Job model.gms Start 06/26/23 16:45:36 36.1.0 r2c0a44a WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\36\gmsprmNT.txt
--- GAMS Parameters defined
    Input C:\Users\dkakkad\AppData\Local\Temp\tmpclxj8ify\model.gms
    Output C:\Users\dkakkad\AppData\Local\Temp\tmpclxj8ify\output.lst
    ScrDir C:\Users\dkakkad\AppData\Local\Temp\tmpclxj8ify\225a\
    SysDir C:\GAMS\36\
    CurDir C:\Users\dkakkad\AppData\Local\Temp\tmpclxj8ify\
    LogOption 3
Licensee: Prof. Ignacio E. Grossmann                     G221207|0002AS-GEN
          Carnegie Mellon University, Dept. of Chemical Engineering  DCE375
          C:\Users\dkakkad\Documents\GAMS\gamslice.txt
          License Admin: Ignacio E. Grossmann, grossmann@cmu.edu           
          Evaluation license: Not for commercial or production use
Processor information: 1 socket(s), 4 core(s), and 8 thread(s) available
GAMS 36.1.0   Copyright (C) 1987-2021 GAMS Development. All rights reserved
--- Starting compilation
--- model.gms(499

In [ ]:
print(model.sol.solver.termination_condition)
print(-model.obj())
print(objs)

In [ ]:
# a = model.y7.extract_values()
# print(a)
# a = np.array(list(model.y7.extract_values().values()))
# print(a)

In [ ]:
for i in links:
    for j in elem:
        if pyo.value(model.f2[i,j]) >= 999:
            print(i,j,pyo.value(model.f2[i,j]))

('BA2', 'BA1') 25 4266.689440647791
('BA2', 'BA4') 12 4124.695704076163
('BA2', 'BA4') 20 1674.474421689506
('BA3', 'BA1') 25 4606.214525555636
('BA3', 'BA14') 1 3114.034571791422
('BA4', 'BA5') 12 3999.999999206157
('BA4', 'BA5') 19 1625.0
('BA4', 'BA6') 12 1338.80806452379
('BA4', 'BA6') 20 2126.453267521244
('BA4', 'BA6') 25 1189.929991618376
('BA7', 'BA9') 23 1233.981999501729
('BA7', 'BA14') 1 9070.203284680087
('BA9', 'BA12') 23 3919.928748333325
('BA9', 'BA12') 25 7535.461379616367
('BA9', 'BA14') 1 3607.757665591585
('BA10', 'BA9') 1 1313.175001421118

In [ ]:
for i in plant:
    sum = 0
    for j in rm:
        if pyo.value(model.f1[j,i]) >= 1e-3:
            sum+=pyo.value(model.f1[j,i])
    print(i,sum)


BA1 9005.545811
BA2 11504.574254009527
BA3 9296.995583421634
BA4 5920.387082953979
BA5 0
BA6 0
BA7 14444.62333911557
BA8 0
BA9 13240.0181998244
BA10 18999.999999999996
BA11 8000.0
BA12 0
BA14 0

In [ ]:
for i in prod:
    sum = 0
    for j in elem:
        sum+=pyo.value(model.f3[i,j])
    print(i, sum)

BA11 19508.575687650133
BA51 3999.999999206157
BA52 2500.0
BA61 1000.0
BA62 2500.0
BA63 3000.0
BA71 4500.000000000001
BA81 3.03341433185343
BA82 500.0
BA83 1000.0
BA101 17596.304502046147
BA111 6603.8701679999995
BA121 9700.360497990092
BA122 2000.0
BA141 14499.999999999998
BA142 1500.0

In [ ]:
for i in objs:
    print(i)

In [ ]:
2.66898811e+08
2.66849462e+08
2.66800796e+08
2.66755743e+08
2.66753605e+08
2.66736721e+08
2.66707978e+08
2.66689619e+08
2.66686783e+08
2.66686325e+08
2.66658755e+08

In [2]:
import timeit

start = timeit.default_timer()

model = ConcreteModel()

model.links = Set(dimen = 2, initialize = links)

#Variables
model.f1 = Var(rm, plant, within = NonNegativeReals, name = 'flow of raw materials', initialize = 0) 
model.f2 = Var(model.links, elem, within = NonNegativeReals, name = 'intermediate flow' , initialize = 0)
model.f3 = Var(prod, elem, within = NonNegativeReals, name = 'flow of elements for each product', initialize = 0)
model.g = Var(prod, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s1 = Var(rm, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s2 = Var(plant, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s3 = Var(prod, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s4 = Var(prod, elem, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s5 = Var(prod, elem, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s6 = Var(rm, plant, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s7 = Var(model.links, elem, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s8 = Var(model.links, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.s9 = Var(model.links, within = NonNegativeReals, name = 'flow of products', initialize = 0)
model.y1 = Var(rm, within = Binary, name = 'flow of products', initialize = 0)
model.y2 = Var(plant, within = Binary, name = 'flow of products', initialize = 0)
model.y3 = Var(prod, within = Binary, name = 'flow of products', initialize = 0)
model.y4 = Var(prod, elem, within = Binary, name = 'flow of products', initialize = 0)
model.y5 = Var(prod, elem, within = Binary, name = 'flow of products', initialize = 0)
model.y6 = Var(rm, plant, within = Binary, name = 'flow of products', initialize = 0)
model.y7 = Var(model.links, elem, within = Binary, name = 'flow of products', initialize = 0)
model.y8 = Var(model.links, within = Binary, name = 'flow of products', initialize = 0)

#parameters
model.RM_analysis = Param(RM_index, initialize = RM_analysis)
model.Availability = Param(rm, initialize = Availability)
model.Rev_prod = Param(prod, initialize = Rev_prod)
model.Rev_mat = Param(rm, initialize = Rev_mat)
model.prod_losses = Param(prod, elem, initialize = prod_losses)
model.mat_deductions = Param(rm, elem, initialize = mat_deductions)
model.elem_price = Param(elem, initialize = elem_price)
model.Capacity = Param(plant, initialize = Capacity)
model.Demand = Param(prod, initialize = Demand)
model.prod_spec_max = Param(prod, elem, initialize = prod_spec_max)
model.prod_spec_min = Param(prod, elem, initialize = prod_spec_min)

#objective
@model.Objective()
def obj(model):
    rev_prod = 0
    rev_mat = 0 
    ML_rm = 0
    ML_prod = 0
    for i in prod:
        for j in elem:
            rev_prod += model.f3[i,j]*model.Rev_prod[i]
            ML_prod += model.f3[i,j]*model.prod_losses[i,j]*model.elem_price[j]
    for i in rm:
        for j in plant:
            rev_mat += model.f1[i,j]*model.Rev_mat[i]
            for k in elem:
                ML_rm += model.f1[i,j]*model.RM_analysis[i, k]*model.mat_deductions[i,k]*model.elem_price[k]

    return (ML_prod - ML_rm - rev_prod - rev_mat)

@model.Constraint(elem, plant)
def MB1(model, elem, plant):
    if plant in identify_plant.keys():
        return sum(model.f1[m,plant]*model.RM_analysis[m,elem] for m in rm) + sum(model.f2[k,elem] for k in model.links if k[1] == plant) == sum(model.f2[k,elem] for k in model.links if k[0] == plant) + sum(model.f3[k,elem] for k in identify_plant[plant])
    else:
        return sum(model.f1[m,plant]*model.RM_analysis[m,elem] for m in rm) + sum(model.f2[k,elem] for k in model.links if k[1] == plant) == sum(model.f2[k,elem] for k in model.links if k[0] == plant)

@model.Constraint(rm)
def avail(model, rm):
#     print(model.Availability.display())
    return sum(model.f1[rm,j] for j in plant) - model.Availability[rm][0] + model.s1[rm] == 0

@model.Constraint(plant)
def capacity(model, plant):
    return sum(model.f1[m,plant] for m in rm) + sum(model.f2[k,m] for m in elem for k in model.links if k[1] == plant) - model.Capacity[plant][0] + model.s2[plant] == 0

@model.Constraint()
def rm_constraint(model):
    return sum((model.f1[m,'BA5']+model.f1[m,'BA6']+model.f1[m,'BA8']+model.f1[m,'BA12']) for m in rm) == 0

@model.Constraint(prod)
def demand(model, prod):
    return model.Demand[prod][0] - sum(model.f3[prod,m] for m in elem) + model.s3[prod] == 0

@model.Constraint(prod)
def prodflow(model, prod):
    return sum(model.f3[prod,m] for m in elem) == model.g[prod]

@model.Constraint(prod, elem)
def prod_min(model, prod, elem):
    return model.prod_spec_min[prod,elem]*model.g[prod] - model.f3[prod, elem] + model.s4[prod,elem] == 0

@model.Constraint(prod, elem)
def prod_max(model, prod, elem):
    return model.f3[prod, elem] - model.prod_spec_max[prod,elem]*model.g[prod] + model.s5[prod,elem] == 0

@model.Constraint(rm, plant)
def lb_f1(model,rm,plant):
    return -model.f1[rm,plant] + model.s6[rm, plant] == 0 

@model.Constraint(lenlink, elem)
def lb_f2(model,lenlink,elem):
    return -model.f2[links[lenlink],elem] + model.s7[links[lenlink],elem] == 0 

@model.Constraint(lenlink)
def constraint1(model, lenlink):
    return -sum(model.f2[links[lenlink],m] for m in elem) + model.s8[links[lenlink]] == 0

@model.Constraint(rm)
def BigM1(model, rm):
    return model.s1[rm] <= 1e8*(1 - model.y1[rm])

@model.Constraint(plant)
def BigM2(model, plant):
    return model.s2[plant] <= 1e8*(1 - model.y2[plant])

@model.Constraint(prod)
def BigM3(model, prod):
    return model.s3[prod] <= 1e8*(1 - model.y3[prod])

@model.Constraint(prod,elem)
def BigM4(model, prod,elem):
    return model.s4[prod,elem] <= 1e8*(1 - model.y4[prod,elem])

@model.Constraint(prod,elem)
def BigM5(model, prod,elem):
    return model.s5[prod,elem] <= 1e8*(1 - model.y5[prod,elem])

@model.Constraint(rm,plant)
def BigM6(model, rm,plant):
    return model.s6[rm,plant] <= 1e8*(1 - model.y6[rm,plant])

@model.Constraint(lenlink,elem)
def BigM7(model,lenlink,elem):
    return model.s7[links[lenlink],elem] <= 1e8*(1 - model.y7[links[lenlink],elem])

@model.Constraint(lenlink)
def BigM8(model, lenlink):
    return model.s8[links[lenlink]] <= 1e8*(1 - model.y8[links[lenlink]])

solver = SolverFactory('gams:gurobi')
model.sol = solver.solve(model,tee= True)
objs.append(-model.obj())


--- Job model.gms Start 06/26/23 16:46:37 36.1.0 r2c0a44a WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\36\gmsprmNT.txt
--- GAMS Parameters defined
    Input C:\Users\dkakkad\AppData\Local\Temp\tmp1t1fm252\model.gms
    Output C:\Users\dkakkad\AppData\Local\Temp\tmp1t1fm252\output.lst
    ScrDir C:\Users\dkakkad\AppData\Local\Temp\tmp1t1fm252\225a\
    SysDir C:\GAMS\36\
    CurDir C:\Users\dkakkad\AppData\Local\Temp\tmp1t1fm252\
    LogOption 3
Licensee: Prof. Ignacio E. Grossmann                     G221207|0002AS-GEN
          Carnegie Mellon University, Dept. of Chemical Engineering  DCE375
          C:\Users\dkakkad\Documents\GAMS\gamslice.txt
          License Admin: Ignacio E. Grossmann, grossmann@cmu.edu           
          Evaluation license: Not for commercial or production use
Processor information: 1 socket(s), 4 core(s), and 8 thread(s) available
GAMS 36.1.0   Copyright (C) 1987-2021 GAMS Development. All rights reserved
--- Starting compilation
--- model.gms(257